# Preparação dos dados
Instalação do Fastai e ferramentas necessárias

In [ ]:
import pandas as pd

from fastai.vision.all import *

import os

#definição da pasta de caminho padrão
path = Path('../input/cassava-leaf-disease-classification')

Obtenção do dataset, neste momento também são excluidas imagens repetidas. As mesmas foram obtidas de notebooks estudados, como [nesse](https://www.kaggle.com/benjibb/resnet50-pretrained-fastai).

In [ ]:
train_df = pd.read_csv(path/'train.csv')
#exclusão de imagens repetidas
train_df = train_df[~train_df['image_id'].isin(['1562043567.jpg', '3551135685.jpg', '2252529694.jpg'])]
train_df.head()

Abaixo são definidos os parâmetros para serem utilizados no datablock para data augmentation, onde o item_tfms ocorre primeiro, e define os parâmetros de transformação para cada uma das imagens em separado, em que foi selecionado um recorte randômico mantendo a proporção e mantendo o mínimo de 75% da imagem. Já o batch_tfms realiza a mesma operação em todas as imagens, defimindo um tamanho de 384px para todas e realizando a normalização.

In [ ]:
item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.))
batch_tfms = [*aug_transforms(size=384, max_warp=0), Normalize.from_stats(*imagenet_stats)]
bs=16

São definidos os getters dos eixos do datablock, onde em x está a pasta onde estão as imagens para treino, e em y estão os labels das categorias.

In [ ]:
def get_x(r):
    return path/'train_images'/r['image_id']

def get_y(r):
    return r['label']

Por fim, é criado o datablock, separando o bloco de imagens e o de categoria, onde é passada a classificação das imagens, sendo de 0 a 4 referidas a plantas doentes, com diferentes categorias, e a 5 referente a planta saudavel. O splitter separa os dados em validação e teste, deixando como 0.2 para validação, e com uma semente randômica definida como 42.

In [ ]:
pets = DataBlock(blocks=(ImageBlock, CategoryBlock),
                 get_x=get_x,
                 get_y=get_y,
                 splitter=RandomSplitter(0.2, seed=42),
                 item_tfms=item_tfms,
                 batch_tfms=batch_tfms)
dataloader = pets.dataloaders(train_df, bs=bs)

In [ ]:
dataloader.show_batch()

# Treinamento do modelo

In [ ]:
#este bloco importa o modelo resnet50 para o local correto, já que não é possível enviar sem a internet ativada
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/resnet50/resnet50.pth' '/root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth'

O modelo escolhido foi o resnet50, devido a testes anteriores utilizando o colab. Foram definidos como métricas de avaliação a acurácia e o error rate.

A forma de treinamento utilizada é o fine_tune, que faz a combinação do fot_one_cycle com o unfreeze, onde primeiramente são geradas epocas com os pesos congelados, e após esse treinamento, os pesos são descongelados para seguir o treinamento. Esse era a sequência padrão utilizada pelo fastai na v1, na v2 foi adicionado o fine_tune para facilitar.

O base_lr é o learning rate base que o modelo vai utilizar. Esse valor foi obtido baseado no resultado de curva descendente de loss antes de ocorrer uma subida brusca, visualizada pelo learn.lr_find() pelo valor 1e-2****

In [ ]:
learn = cnn_learner(dataloader, resnet50,
                    loss_func = LabelSmoothingCrossEntropy(),
                    cbs=[MixUp()],
                    metrics=[error_rate,accuracy])
learn.fine_tune(8,freeze_epochs = 2,base_lr=1e-2)

Abaixo estou salvando o modelo com os pesos treinados em um arquivo, para realizar o download para minha máquina

In [ ]:
#learn.model_dir = '/kaggle/working'
#learn.save('kaggle_resnet50')
#learn.load('../input/kaggleresnet50/kaggle_resnet50.pth')

In [ ]:
learn = learn.to_native_fp32()

# Inferência e submissão
Nessa parte é preparada a submissão, abaixo é lido o csv de submissão preparado 

In [ ]:
sample_df = pd.read_csv(path/'sample_submission.csv')
sample_df.head()

As imagens disponíveis para teste são preditas utilizando como base o modelo treinado

In [ ]:
test_data_path = sample_df['image_id'].apply(lambda x: path/'test_images'/x)
tst_dl = learn.dls.test_dl(test_data_path)
predictions = learn.tta(dl = tst_dl, n=10)



In [ ]:
sample_df['label'] = np.argmax(predictions[0],axis=1)
sample_df

Por fim, é criado o submission.csv com as predições geradas anteriormente

In [ ]:
sample_df.to_csv('submission.csv',index=False)